In [1]:
# utility from fink-science
from fink_science.utilities import concat_col

# user-defined function from the current folder
from processor import rfscore_pca

In [2]:
df = spark.read.format('parquet').load('/media/biswajit/drive/Kilonova_datasets/20191101')

In [3]:
# Required alert columns
what = ['jd', 'fid', 'magpsf', 'sigmapsf']

# Use for creating temp name
prefix = 'c'
what_prefix = [prefix + i for i in what]

# Concatenate historical + current measurements
for colname in what:
    df = concat_col(df, colname, prefix=prefix)
#args = [df.col(i) for i in what_prefix]

In [4]:
#import pickle 
pkl_filename = "models/pickle_model.pkl"
#with open(pkl_filename, 'rb') as file:
#    pickle_model = pickle.load(file)

In [5]:
#calculating probability that the alert is a kilonova
from pyspark.sql import functions as F
df_change = df.withColumn('pca_score', rfscore_pca('cjd', 'cfid', 'cmagpsf','csigmapsf', F.lit("models/pickle_model.pkl")) )

In [6]:
#first 20 prbabilities of alert being a kilonova
df_change.select(['pca_score']).show()

+-------------------+
|          pca_score|
+-------------------+
|                0.0|
|0.17748262818840746|
|                0.0|
|                0.0|
|                0.0|
| 0.1940903778117784|
|                0.0|
|                0.0|
| 0.0545446320104014|
|                0.0|
| 0.1909867516543953|
|                0.0|
| 0.1909867516543953|
|0.25635052643671463|
|                0.0|
|                0.0|
|0.19293637671454517|
|0.27040470900787916|
| 0.1951857883239929|
|                0.0|
+-------------------+
only showing top 20 rows

